In [1]:
%load_ext autoreload
%autoreload 2
import torch

import os
import sys
module_path = os.path.abspath(os.path.join('../../../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load models

In [2]:
from classification.models.crnn.model import AudioCRNN
from classification.models.SpectrogramCNN import SpectrogramCNN
from classification.models.DeepRecursiveCNN import DeepRecursiveCNN
from classification.models.M5 import M5

In [3]:
models = {}

In [4]:
state_dict_path = '/nfs/students/summer-term-2020/project-4/data/models/SpectrogramBasedCNN.pt'

model = SpectrogramCNN()
model.load_state_dict(torch.load(state_dict_path))
model = model.cuda()
model = model.eval()

models[model.__class__.__name__] = model

In [5]:
model_state_dict_path = "/nfs/students/summer-term-2020/project-4/yan/models/best_model_state_dict.pt"
model = DeepRecursiveCNN()
model.load_state_dict(torch.load(model_state_dict_path))
model = model.eval()

models[model.__class__.__name__] = model

In [44]:
res_path = '/nfs/students/summer-term-2020/project-4/data/models/crnn.pth'
checkpoint = torch.load(res_path)
model = AudioCRNN(checkpoint['classes'], checkpoint['config'])
model.load_state_dict(checkpoint['state_dict'])
model = model.eval().cuda()

models[model.__class__.__name__] = model

In [6]:
loaded_dict = torch.load("/nfs/homedirs/herrmanp/project-4/experiments/notebooks/pascal/adv_totaly_hig_50epochs.pt")
model = M5(loaded_dict["hparams"])
model.load_state_dict(loaded_dict["state_dict"])
model = model.eval().cuda()

models[model.__class__.__name__] = model

# Load Data

In [7]:
from datasets.datahandler import DatasetHandler
datasetHandler = DatasetHandler()

In [8]:
for model_name in models:
    #datasetHandler.load(models[model_name], 'training')
    datasetHandler.load(models[model_name], 'validation')

Loading cached val data from /nfs/students/summer-term-2020/project-4/data/data_8k


## FGSM

In [9]:
from attacks.pgd import ProjectedGradientDescent

attack_parameters = {'epsilon': 0.001, 'num_iter': 1}
attacks = {}

for model_name in models:
    valid_loader = models[model_name].getDataLoader('validation', batch_size=3)
    fgsm = ProjectedGradientDescent(model, valid_loader, attack_parameters, save_samples=False)
    fgsm.attack()
    attacks[model_name] = fgsm
    
for model_name in attacks:
    print(model_name)
    print(attacks[model_name].report())
    print()

100%|██████████| 563/563 [00:05<00:00, 100.89it/s]


SpectrogramCNN
Attack-Parameters:	{'epsilon': 0.001, 'num_iter': 1}
Early stopping: 	False (-1)

Successfully attacked:	23
Total attacked: 	823
Total processed:	1687

Success-Rate: 		0.03
Perturbed Accurracy: 	0.47
None

DeepRecursiveCNN
Attack-Parameters:	{'epsilon': 0.001, 'num_iter': 1}
Early stopping: 	False (-1)

Successfully attacked:	23
Total attacked: 	823
Total processed:	1687

Success-Rate: 		0.03
Perturbed Accurracy: 	0.47
None

M5
Attack-Parameters:	{'epsilon': 0.001, 'num_iter': 1}
Early stopping: 	False (-1)

Successfully attacked:	35
Total attacked: 	1356
Total processed:	1687

Success-Rate: 		0.03
Perturbed Accurracy: 	0.78
None



## PGD 

In [9]:
from attacks.pgd import ProjectedGradientDescent

attack_parameters = {'epsilon': 0.001, 'num_iter': 3}
attacks = {}

for model_name in models:
    valid_loader = models[model_name].getDataLoader('validation', batch_size=3)
    fgsm = ProjectedGradientDescent(model, valid_loader, attack_parameters, save_samples=False)
    fgsm.attack()
    attacks[model_name] = fgsm
    
for model_name in attacks:
    print(model_name)
    attacks[model_name].report()
    print()

100%|██████████| 563/563 [00:10<00:00, 51.20it/s]

M5
Attack-Parameters:	{'epsilon': 0.001, 'num_iter': 3}
Early stopping: 	False (-1)

Successfully attacked:	92
Total attacked: 	1356
Total processed:	1687

Success-Rate: 		0.07
Perturbed Accurracy: 	0.75



In [ ]:
# pgd.showAdversarialExample(sr=FIXED_SAMPLE_RATE, target_class=1)

## Volume attacks

In [33]:
from attacks.volume import VolumeAttack

attack_parameters = {'epsilon': 0.1, 'num_iter': 10}
attacks = {}

for model_name in models:
    valid_loader = models[model_name].getDataLoader('validation', batch_size=10)
    attack = VolumeAttack(model, valid_loader, attack_parameters, save_samples=False)
    attack.attack()
    attacks[model_name] = attack
    
for model_name in attacks:
    print(model_name)
    attacks[model_name].report()
    print()

100%|██████████| 169/169 [00:25<00:00,  6.67it/s]

SpectrogramCNN
Attack-Parameters:	{'epsilon': 0.1, 'num_iter': 10}
Early stopping: 	False (-1)

Successfully attacked:	129
Total attacked: 	823
Total processed:	1687

Success-Rate: 		0.16
Perturbed Accurracy: 	0.41

DeepRecursiveCNN
Attack-Parameters:	{'epsilon': 0.1, 'num_iter': 10}
Early stopping: 	False (-1)

Successfully attacked:	129
Total attacked: 	823
Total processed:	1687

Success-Rate: 		0.16
Perturbed Accurracy: 	0.41

M5
Attack-Parameters:	{'epsilon': 0.1, 'num_iter': 10}
Early stopping: 	False (-1)

Successfully attacked:	96
Total attacked: 	1356
Total processed:	1687

Success-Rate: 		0.07
Perturbed Accurracy: 	0.75



## Interpolation attack

In [60]:
from attacks.interpolation import InterpolationAttack

attacks = {}

for model_name in fast_model:
    valid_loader = models[model_name].getDataLoader('validation', batch_size=20)
    test_sound = 0.2*next(iter(valid_loader))['audio'][0]
    attack_parameters = {'epsilon': 0.2, 'num_iter': 10, 'overlay_sound': test_sound}
    attack = InterpolationAttack(model, valid_loader, attack_parameters, save_samples=False)
    attack.attack()
    attacks[model_name] = attack

100%|██████████| 85/85 [00:21<00:00,  3.90it/s]


In [59]:
for model_name in attacks:
    print(model_name)
    attacks[model_name].report()
    print()

M5
Attack-Parameters:	{'epsilon': 0.2, 'num_iter': 10, 'overlay_sound': tensor([-0.0449, -0.0255, -0.0231,  ..., -0.0050,  0.0054, -0.0017])}
Early stopping: 	False (-1)

Successfully attacked:	7
Total attacked: 	1356
Total processed:	1687

Success-Rate: 		0.01
Perturbed Accurracy: 	0.8



## Time stretching attack 

In [177]:
from attacks.speed import TimeStretchAttack

attack_parameters = {'num_iter': 30, 'lower': 0.5, 'upper': 1.5}
attacks = {}

for model_name in fast_model:
    valid_loader = models[model_name].getDataLoader('validation', batch_size=30)
    attack = TimeStretchAttack(model, valid_loader, attack_parameters)
    attack.attack()
    attacks[model_name] = attack

100%|██████████| 57/57 [00:44<00:00,  1.28it/s]


In [174]:
#attacks['M5'].showAdversarialExample(target_class=1)

In [178]:
#tsa.showAdversarialExample(target_class=0)
for model_name in attacks:
    print(model_name)
    attacks[model_name].report()
    print()

M5
Attack-Parameters:	{'num_iter': 30, 'lower': 0.5, 'upper': 1.5}
Early stopping: 	False (-1)

Successfully attacked:	108
Total attacked: 	1356
Total processed:	1687

Success-Rate: 		0.08
Perturbed Accurracy: 	0.74



## Pitch attack

In [189]:
from attacks.attack import Attack

from torchaudio import functional as AF
import torch.nn.functional as F
import torchaudio
import math
import torch
import librosa

class PitchAttack(Attack):

    def attackSample(self, x, y, num_iter=1, lower=-1, upper=1):
        n_steps_search_range = torch.arange(lower, upper, (upper-lower)/num_iter)
        losses = []
        stretched_inputs = []
        
        with torch.no_grad():
            x_original = x['audio'].clone().cpu().numpy()
            
            for n_steps in n_steps_search_range:
                ## Problem: librosa does not support batch > 1 
                x['audio'] = librosa.effects.pitch_shift(x_original, sr=x['sample_rate'], n_steps=n_steps)
                stretched_inputs.append(x['audio'])
                losses.append(F.nll_loss(self.model(x), y, reduction='none'))
                 
        best_rates = torch.stack(losses).argmax(0)

        for x_i in range(len(best_rates)): # here is potential to further vectorize 
            x['audio'][x_i] = stretched_inputs[best_rates[x_i]][x_i].clamp(-1, 1)
        return x

In [12]:
fast_model = {'M5': models['M5']}

In [190]:
#from attacks.pitch import PitchAttack

attack_parameters = {'num_iter': 5, 'lower': 0, 'upper': 10}
attacks = {}

for model_name in fast_model:
    valid_loader = models[model_name].getDataLoader('validation', batch_size=30)
    attack = PitchAttack(model, valid_loader, attack_parameters)
    attack.attack()
    attacks[model_name] = attack

  0%|          | 0/57 [00:00<?, ?it/s]


ParameterError: Invalid shape for monophonic audio: ndim=2, shape=(24, 80000)

In [6]:
#pa.showAdversarialExample(target_class=0)

In [9]:
pa.report()

Attack-Parameters:	{'num_iter': 5, 'lower': 0, 'upper': 10}
Early stopping: 	True (10)

Successfully attacked:	10
Total attacked: 	87
Total processed:	98

Success-Rate: 		0.11
Perturbed Accurracy: 	0.79
